In [1]:
import mysql.connector

In [2]:
config = {
   'user': 'root',
   'password': '123123',
   'host': 'localhost',
   'port': 3306,
   'database': 'plant_nursery',
   'raise_on_warnings': True                    
}

In [3]:
db_connection = mysql.connector.connect(**config)

In [4]:
#open a cursor (iterator) for stepping through the tuples in the resultset of an SQL query 
cur = db_connection.cursor()

In [5]:
answer = True
while answer == True:
    print("Welcome to the Plant Nursery! \n \n")
    print("Please choose one of the options: \n")
    print("1.)  Print all of the plants that are available at the plant nursery")
    print("2.)  List the unique common names of all the vegetables currently planted \n")
    print("3.)  List the species name and number of varieties for each plant species for all the plants. \n")
    print("4.)  Find the total number of pots currently holding a plant with a given common name. \n")
    print("5.)  Find the histogram (value and count) of volumes of pots with germinated plants with a \n")
    print("     given common name. \n")
    print("6.)  Find the most populous species among the pots with germinated plants. \n")
    print("7.)  Find the oldest vegetable plant(s) among those that germinated during the previous month. \n")
    print("8.)  Find the germinated plant(s) that received the most daily water (averaged over their age). \n")
    print("9.)  Find the number of planted herbs that received more food than the average amount of food \n")
    print("     received by the germinated vegetables. \n")
    print("10.)  Find the germinated flowers that received more daily ambient light in a given month than the \n")
    print("     previous month. \n")
    print("0.)  Exit the program \n \n")
    
    userInput = input("Enter your option: ")
    print('\n')
    
    if userInput == '1':
        query = "select species_name, common_name, variety, type, germ_days from plants"
        cur.execute(query)
        print("{:25} {:20} {:25} {:20} {:10}".format('|Species Name:|', '|Common Name:|', \
                                                          '|Variety:|', '|Type:|',\
                                                          '|Germ. Days:|'))
        print("_________________________________________________________________________________________")
        for(species,common,var,type,germ) in cur:
            print("{:<25} {:<20} {:<25} {:<20} {:<10}".format(species,common,var,type,germ))
        print('\n')
        input("Press Enter to continue...")
        print('\n')

    
    if userInput == '2':
        query = "select distinct common_name from plants A, potted B where A.species_name = B.species_name"
        cur.execute(query)
        print("Common Names:")
        print("___________________")
        for result in cur:
            print("{:<10}".format(result[0]))
        print('\n')
        input("Press Enter to continue...")
        print('\n')
        
    elif userInput == '3':
        query = "select distinct species_name, count(variety) from plants group by species_name"
        cur.execute(query)
        print("{:25} {:10}".format('Species Name:', '# of Varieties:'))
        print("_________________________________________________")
        for (common_name, count) in cur:
            print("{:<25} {:<10}".format(common_name, count))
        print('\n')
        input("Press Enter to continue...")
        print('\n')
        
    elif userInput == '4':
        givenName = input("Enter common name that is inside a plant (Capitalize the first letter): ")
        query = "select common_name, count(common_name) from potted natural inner join plants \
        where common_name = '" + givenName + "' group by common_name"
        cur.execute(query)
        print("\n")
        print("{:25} {:15}".format('Common Name:', '# of Potted Plants:'))
        print("__________________________________________________")
        for(common_name, count) in cur:
            print("{:<25} {:<15}".format(common_name, count))
        print('\n')
        input("Press Enter to continue...")
        print('\n')
        
    elif userInput == '5':
        givenName = input("Enter common name that is inside a plant (Capitalize the first letter): ")
        query = "select volume, count(volume) from (potted natural inner join plants) natural inner join pots \
        where germ_date is not null and common_name = '" + givenName + "' group by volume"
        cur.execute(query)
        print("\n")
        print("{:15} {:15}".format('Volume:', 'Count:'))
        print("_____________________________________")
        for(volume, count) in cur:
            print("{:<15} {:<15}".format(volume, count))
        print('\n')
        input("Press Enter to continue...")
        print('\n')
        
    elif userInput == '6':
        query = "select species_name, count(species_name) as populous from potted natural inner join pots \
        where germ_date is not null group by species_name having count(species_name) = \
        (select count(species_name) as p from potted natural inner join pots where \
        germ_date is not null group by species_name order by p desc limit 1)"
        cur.execute(query)
        print("{:25} {:20}".format('Species Name:', '# of Germinated Species:'))
        print("__________________________________________________________")
        for(volume, count) in cur:
            print("{:<25} {:<20}".format(volume, count))     
        print('\n')
        input("Press Enter to continue...")
        print('\n')
        
    elif userInput == '7':
        query = "select variety, pot_id from potted natural inner join pots natural join plants \
        where type = 'vegetable' and germ_date is not null and germ_date <= now() - interval 1 month \
        and now() - germ_date = (select now() - germ_date from potted natural inner join \
        pots natural join plants where germ_date is not null order by now() - germ_date desc limit 1)"
        cur.execute(query)
        print("{:20}{:20}".format('Oldest Plant(s):', 'Pot-ID'))
        print("________________________________")
        for (var, id) in cur:
            print("{:<20}{:<20}".format(var, id)) 
        print('\n')
        input("Press Enter to continue...")
        print('\n')
        
    elif userInput == '8':
        query = "select variety, avg(watering) as water from (potted natural inner join pots) \
        natural inner join logs where germ_date is not null group by variety having avg(watering) \
        = (select avg(watering) as w from (potted natural inner join pots) natural inner join logs \
        where germ_date is not null group by variety order by w desc limit 1)"
        cur.execute(query)
        print("{:30} {:15}".format('Plant(s):', 'Amount of Water:'))
        print("_______________________________________________________")
        for (plants, water) in cur:
            print("{:<30} {:<15}".format(plants, water))   
        print('\n')
        input("Press Enter to continue...")
        print('\n')

        
    elif userInput == '9':
        query = "select count(*) from (plants natural inner join (potted natural inner join pots)) \
        natural inner join logs where type = 'herb' and feeding > (select avg(feeding) \
        from (plants natural inner join (potted natural inner join pots)) natural inner join logs \
        where type = 'vegetable')"
        cur.execute(query)
        print("{:30}".format('# of Herbs that Received More Food than Vegetables'))
        print("_______________________________________________________")
        for result in cur:
            print("{:<30}".format(result[0])) 
        print('\n')
        input("Press Enter to continue...")
        print('\n')
        
        
    elif userInput == '10':
        print("Not Implemented \n")
    elif userInput == '0':
        print("Have a nice day!")
        answer = False

    
    
    

Welcome to the Plant Nursery! 
 

Please choose one of the options: 

1.)  Print all of the plants that are available at the plant nursery
2.)  List the unique common names of all the vegetables currently planted 

3.)  List the species name and number of varieties for each plant species for all the plants. 

4.)  Find the total number of pots currently holding a plant with a given common name. 

5.)  Find the histogram (value and count) of volumes of pots with germinated plants with a 

     given common name. 

6.)  Find the most populous species among the pots with germinated plants. 

7.)  Find the oldest vegetable plant(s) among those that germinated during the previous month. 

8.)  Find the germinated plant(s) that received the most daily water (averaged over their age). 

9.)  Find the number of planted herbs that received more food than the average amount of food 

     received by the germinated vegetables. 

10.)  Find the germinated flowers that received more daily ambient 



Welcome to the Plant Nursery! 
 

Please choose one of the options: 

1.)  Print all of the plants that are available at the plant nursery
2.)  List the unique common names of all the vegetables currently planted 

3.)  List the species name and number of varieties for each plant species for all the plants. 

4.)  Find the total number of pots currently holding a plant with a given common name. 

5.)  Find the histogram (value and count) of volumes of pots with germinated plants with a 

     given common name. 

6.)  Find the most populous species among the pots with germinated plants. 

7.)  Find the oldest vegetable plant(s) among those that germinated during the previous month. 

8.)  Find the germinated plant(s) that received the most daily water (averaged over their age). 

9.)  Find the number of planted herbs that received more food than the average amount of food 

     received by the germinated vegetables. 

10.)  Find the germinated flowers that received more daily ambien

Enter your option: 


Welcome to the Plant Nursery! 
 

Please choose one of the options: 

1.)  Print all of the plants that are available at the plant nursery
2.)  List the unique common names of all the vegetables currently planted 

3.)  List the species name and number of varieties for each plant species for all the plants. 

4.)  Find the total number of pots currently holding a plant with a given common name. 

5.)  Find the histogram (value and count) of volumes of pots with germinated plants with a 

     given common name. 

6.)  Find the most populous species among the pots with germinated plants. 

7.)  Find the oldest vegetable plant(s) among those that germinated during the previous month. 

8.)  Find the germinated plant(s) that received the most daily water (averaged over their age). 

9.)  Find the number of planted herbs that received more food than the average amount of food 

     received by the germinated vegetables. 

10.)  Find the germinated flowers that receiv